In [1]:
import numpy as np
import torch
from torch import nn
import random

from torch.utils.data import Dataset, DataLoader

from tqdm import tqdm

from torch.utils.tensorboard import SummaryWriter

import os

from datetime import datetime

In [2]:
train_betas_path = '../data/processed_data/subj01/nsd_train_fmriavg_nsdgeneral_sub1.npy'
train_caps_embed_path = '../data/caps_embeds/train_caps_embeds_sub1.npy'
train_caps_embed_neg_path = '../data/caps_embeds/train_caps_embeds_negative_sub1.npy'

train_betas = np.load(train_betas_path)
# train_caps_embed = np.load(train_caps_embed_path)
# train_caps_embed_neg = np.load(train_caps_embed_neg_path)

In [3]:
train_betas.shape

(8859, 15724)

In [4]:
test_betas_path = '../data/processed_data/subj01/nsd_test_fmriavg_nsdgeneral_sub1.npy'
test_caps_embed_path = '../data/caps_embeds/test_caps_embeds_sub1.npy'
test_caps_embed_neg_path = '../data/caps_embeds/test_caps_embeds_negative_sub1.npy'

# test_betas = np.load(test_betas_path)
# test_caps_embed = np.load(test_caps_embed_path)
# test_caps_embed_neg = np.load(test_caps_embed_neg_path)

In [5]:
class CustomDataset(Dataset):
    def __init__(self, betas_path, embeds_path, neg_embeds_path):
        self.betas = torch.from_numpy(np.load(betas_path)).float().cuda()
        self.embeds = torch.from_numpy(
            np.squeeze(np.load(embeds_path))
            ).float().cuda()
        self.embeds_neg = torch.from_numpy(
            np.squeeze(np.load(neg_embeds_path))
            ).float().cuda()

    def __len__(self):
        return len(self.betas)

    def __getitem__(self, index):
        return self.betas[index], self.embeds[index], self.embeds_neg[index]


class CustomDataLoader:
    def __init__(self, dataset, batch_size):
        self.data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True)

    def __iter__(self):
        return iter(self.data_loader)

    def __len__(self):
        return len(self.data_loader)

batch_size = 64

train_dataset = CustomDataset(train_betas_path,
                        train_caps_embed_path,
                        train_caps_embed_neg_path)
train_data_loader = CustomDataLoader(train_dataset, batch_size)

test_dataset = CustomDataset(test_betas_path,
                              test_caps_embed_path,
                              test_caps_embed_neg_path)
test_data_loader = CustomDataLoader(test_dataset, batch_size)

for beta, embed, neg_embed in train_data_loader:
    print(f"beta      {beta.shape}")
    print(f"embed     {embed.shape}")
    print(f"neg_embed {neg_embed.shape}")
    break

beta      torch.Size([64, 15724])
embed     torch.Size([64, 1280])
neg_embed torch.Size([64, 1280])


In [6]:
input_dim = 15724
print('input_dim:', input_dim)
ouput_dim = 1280
print('output_dim:', ouput_dim)

input_dim: 15724
output_dim: 1280


In [7]:
class RegressionModel(nn.Module):
    def __init__(self, input_channels, output_channels):
        super().__init__()
        self.linear_regression = nn.Linear(in_features=input_channels,
                               out_features=output_channels)
        self.layer1 = 2**10 * 4
        self.layer2 = 2**10 * 5

        self.mlp = nn.Sequential(
            nn.Linear(in_features=input_channels, 
                      out_features=self.layer1),
            nn.LeakyReLU(0.05, inplace=True),    
            nn.Linear(in_features=self.layer1, 
                      out_features=output_channels),
        )


    def forward(self, x):
        return self.mlp(x)

model_embed = RegressionModel(input_dim, ouput_dim).cuda()
model_neg = RegressionModel(input_dim, ouput_dim).cuda()

In [8]:
next(model_embed.parameters()).dtype

torch.float32

In [9]:
import torch.nn.functional as F

class CosineSimilarityLoss(nn.Module):
    def __init__(self):
        super(CosineSimilarityLoss, self).__init__()

    def forward(self, y_true, y_pred):
        cosine_sim = F.cosine_similarity(y_true, y_pred, dim=1)
        loss = 1 - cosine_sim.mean()
        return loss

In [10]:
cos_loss = CosineSimilarityLoss()
mse_loss = nn.MSELoss(reduction='mean')
optim_embed = torch.optim.Adam(params=model_embed.parameters(), lr=0.0001, weight_decay=1)
optim_neg = torch.optim.Adam(params=model_neg.parameters(), lr=0.0001, weight_decay=1)

/home/ilgiz/dev/boldbrain/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [11]:
run_name = 'mlp_only_cos' #+ datetime.now().strftime("%Y-%m-%d_%H:%M")
writer = SummaryWriter(os.path.join('runs', run_name))

In [12]:
import torch
from tqdm import tqdm

num_epochs = 100
iter_index = 0
min_test_loss = 1e15
best_model_path = f'../models/{run_name}.pth'
test_iter_index = 0

for epoch in tqdm(range(num_epochs)):
    model_embed.train()
    model_neg.train()

    for betas, embeds, neg_embeds in train_data_loader:
        iter_index += 1
        pred_embeds = model_embed(betas)
        pred_neg_embeds = model_neg(betas)

        loss_embed = mse_loss(pred_embeds, embeds)
        optim_embed.zero_grad()
        loss_embed.backward()
        optim_embed.step()
        
        loss_neg = mse_loss(pred_neg_embeds, neg_embeds)
        optim_neg.zero_grad()
        loss_neg.backward()
        optim_neg.step()

        writer.add_scalars('train/losses', {
            'embed': loss_embed.item(),
            'negat': loss_neg.item(),
        }, iter_index)

    test_loss = 0
    
    model_embed.eval()
    model_neg.eval()

    # if epoch % 10 == 0:
    with torch.inference_mode():
        n_batches = 0
        for betas, embeds, neg_embeds in test_data_loader:
            test_iter_index += 1
            pred_embeds = model_embed(betas)
            pred_neg_embeds = model_neg(betas)

            # loss_embed = mse_loss(pred_embeds, embeds)
            # loss_neg = mse_loss(pred_neg_embeds, neg_embeds)

            cos_embed = cos_loss(pred_embeds, embeds)
            cos_neg = cos_loss(pred_neg_embeds, neg_embeds)

            writer.add_scalars('test/losses', {
                'embed': loss_embed.item(),
                'negat': loss_neg.item(),
            }, test_iter_index)

            writer.add_scalars('test/cos_sim', {
                'cos_embed': cos_embed.item(),
                'cos_neg': cos_neg.item(),
            }, test_iter_index)

            n_batches += 1
            test_loss += loss_embed.item() + loss_neg.item()
    test_loss /= n_batches
    if test_loss < min_test_loss:
        min_test_loss = test_loss
        print(f'Model saved with loss: {test_loss}')
        torch.save({
            'model_embed_state_dict': model_embed.state_dict(),
            'model_neg_state_dict': model_neg.state_dict(),
            'epoch': epoch,
            'min_test_loss': min_test_loss,
        }, best_model_path)


  0%|          | 0/100 [00:00<?, ?it/s]

Model saved with loss: 52.54768180847168


  1%|          | 1/100 [00:12<20:40, 12.53s/it]

Model saved with loss: 43.72822570800781


  2%|▏         | 2/100 [00:25<20:38, 12.64s/it]

...

Model saved with loss: 1.2736429870128632


 63%|██████▎   | 63/100 [12:47<07:11, 11.66s/it]

Model saved with loss: 1.2558721601963043


 77%|███████▋  | 77/100 [15:28<04:37, 12.06s/it]


KeyboardInterrupt: 